In order to successfully complete this assignment you must do the required reading, watch the provided videos and complete all instructions.  The embedded survey form must be entirely filled out and submitted on or before **11:59pm**.  Students must come to class the next day prepared to discuss the material covered in this assignment. 

---

# PCA 30: MPI Errors

### Goals for today's pre-class assignment 

</p>

1. At its core, MPI is Simple
2. Error Handling In MPI
3. More on MPI Tags
4. Assignment wrap-up


# 1. At its core, MPI is Simple
Remember that most MPI programs can be written using just these six functions, only two of which are non-trivial:
* MPI_INIT
* MPI_COMM_SIZE
* MPI_COMM_RANK
* MPI_SEND
* MPI_RECV
* MPI_FINALIZE

If you can remember these functions the rest is easy.

# 2. Error Handling In MPI

You should also do error handling in MPI some things to note are:

* By default, an error causes all processes to abort. (sometimes the message is not that helpful).
* The user can cause routines to return (with an error code) instead.

Parallel programs are hard to debug. It is always a good habit to activly manage errors.  


&#9989; **<font color=red>DO THIS:</font>** Find a reference that explains how to handle errors in MPI.  

&#9989; **<font color=red>QUESTION:</font>** What is the reference link you found on handling errors in MPI?

https://www.mpi-forum.org/docs/mpi-1.1/mpi-11-html/node148.html

&#9989; **<font color=red>DO THIS:</font>** Review the following code we got to work on the HPCC last week.  Add code to include errors handling. 

```c++
    #include <stdio.h>
    #include <stdlib.h>
    #include <mpi.h>
    #include <unistd.h>
    
    #define max_nodes 264
    #define str_length 50
    
    void checkMPIError(int errCode, const char* msg) {
        if (errCode != MPI_SUCCESS) {
            char errStr[MPI_MAX_ERROR_STRING];
            int errLen;
            MPI_Error_string(errCode, errStr, &errLen);
            fprintf(stderr, "MPI ERROR in %s: %s\n", msg, errStr);
            MPI_Abort(MPI_COMM_WORLD, errCode);
        }
    }
    
    int main(int argc, char **argv) {
        int proc, rank, size, namelen;
        int ids[max_nodes];
        char hostname[max_nodes][str_length];  // Corrected array dimension
        char p_name[str_length];
    
        MPI_Status status;
    
        int err = MPI_Init(&argc, &argv);
        checkMPIError(err, "MPI_Init");
    
        // Set error handler to return errors instead of aborting
        MPI_Errhandler_set(MPI_COMM_WORLD, MPI_ERRORS_RETURN);
    
        err = MPI_Comm_rank(MPI_COMM_WORLD, &rank);
        checkMPIError(err, "MPI_Comm_rank");
    
        err = MPI_Comm_size(MPI_COMM_WORLD, &size);
        checkMPIError(err, "MPI_Comm_size");
    
        err = MPI_Get_processor_name(p_name, &namelen);
        checkMPIError(err, "MPI_Get_processor_name");
    
        if (rank == 0) {
            printf("Hello From: %s I am the receiving processor %d of %d\n", p_name, rank + 1, size);
            for (proc = 1; proc < size; proc++) {
                err = MPI_Recv(hostname[proc], str_length, MPI_CHAR, proc, 1, MPI_COMM_WORLD, &status);
                checkMPIError(err, "MPI_Recv (hostname)");
    
                err = MPI_Recv(&ids[proc], 1, MPI_INT, proc, 2, MPI_COMM_WORLD, &status);
                checkMPIError(err, "MPI_Recv (id)");
    
                printf("Hello From: %-20s I am processor %d of %d\n", hostname[proc], ids[proc] + 1, size);
            }
        } else {
            srand(rank);
            int t = rand() % 10 + 1;
            sleep(t);
    
            err = MPI_Send(p_name, str_length, MPI_CHAR, 0, 1, MPI_COMM_WORLD);
            checkMPIError(err, "MPI_Send (hostname)");
    
            err = MPI_Send(&rank, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
            checkMPIError(err, "MPI_Send (rank)");
        }
    
        err = MPI_Finalize();
        checkMPIError(err, "MPI_Finalize");
    
        return 0;
}

```

&#9989; **<font color=red>QUESTION:</font>** Where you able to figure out how to get the error handling to work?

error handling in MPI works only if you explicitly set the error handler to return errors instead of aborting the program, which is not the default behavior.

&#9989; **<font color=red>QUESTION:</font>** If so, what was most challenging/Interesting? If not, where did you get stuck?

What I found most interesting (and a bit tricky) when getting MPI error handling to work was understanding MPI's default behavior and how deceptively easy it is to assume errors will just return codes like in regular C — when in fact, they often don't unless you explicitly change the error handling mode.

# 3. More on MPI Tags
* Messages are sent with an accompanying user-defined integer tag, to assist the receiving process in identifying the message.
* Messages can be screened at the receiving end by specifying a specific tag, or not screened by specifying MPI_ANY_TAG as the tag in a receive.
* Some non-MPI message-passing systems have called tags “message types”. MPI calls them tags to avoid confusion with datatypes. 

&#9989; **<font color=red>QUESTION:</font>** How many different tags are used in the above example?

2

&#9989; **<font color=red>QUESTION:</font>** How are the tags being used?

One for sending and receivng the hostname and one for sending and receiving the rank

----

<a name="Assignment_wrap-up"></a>
# Assignment wrap-up

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credits for the assignment!**

[Direct Link to Survey Form](https://cmse.msu.edu/cmse401-pc-survey)


&#9989; **<font color=red>Assignment-Specific QUESTION:</font>**  No assignment specific question. 

Put your answer to the above question here

&#9989; **<font color=red>QUESTION:</font>**  Summarize what you did in this assignment.

Learned about MPI error handling

&#9989; **<font color=red>QUESTION:</font>**  What questions do you have, if any, about any of the topics discussed in this assignment after working through the jupyter notebook?

Can you go through an example of error handling just for syntax purposes

&#9989; **<font color=red>QUESTION:</font>**  How well do you feel this assignment helped you to achieve a better understanding of the above mentioned topic(s)?

Well

&#9989; **<font color=red>QUESTION:</font>** What was the **most** challenging part of this assignment for you? 

Changing the code to support error handling

&#9989; **<font color=red>QUESTION:</font>** What was the **least** challenging part of this assignment for you? 

Find a source for learning error handling

&#9989; **<font color=red>QUESTION:</font>**  What kind of additional questions or support, if any, do you feel you need to have a better understanding of the content in this assignment?

None

&#9989; **<font color=red>QUESTION:</font>**  Do you have any further questions or comments about this material, or anything else that's going on in class?

None

&#9989; **<font color=red>QUESTION:</font>** Approximately how long did this pre-class assignment take?

30 minutes

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://cmse.msu.edu/cmse401-pc-survey" 
	width="100%" 
	height="500px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)


---------
### Congratulations, we're done!

To get credit for this assignment you must fill out and submit the above survey from on or before the assignment due date.

Written by Dr. Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----

----